In [144]:
# Bresenham line algorithm
# https://gist.github.com/flags/1132363

# define a class for the Bresenham algorithm
class bresenham:
    # the constructor initializes the start and end points, the path of the line, and determines whether the line is steep or not
    def __init__(self, start, end):
        self.start = list(start)
        self.end = list(end)
        self.path = []

        self.steep = abs(self.end[1]-self.start[1]) > abs(self.end[0]-self.start[0])

        # if the line is steep, swap the x and y coordinates of the start and end points
        if self.steep:
            self.start = self.swap(self.start[0],self.start[1])
            self.end = self.swap(self.end[0],self.end[1])

        # if the start x coordinate is greater than the end x coordinate, swap the start and end points
        if self.start[0] > self.end[0]:
            _x0 = int(self.start[0])
            _x1 = int(self.end[0])
            self.start[0] = _x1
            self.end[0] = _x0

            # also swap the y coordinates of the start and end points
            _y0 = int(self.start[1])
            _y1 = int(self.end[1])
            self.start[1] = _y1
            self.end[1] = _y0

        # calculate the differences in x and y coordinates and the error
        dx = self.end[0] - self.start[0]
        dy = abs(self.end[1] - self.start[1])
        error = 0
        derr = dy/float(dx)

        # determine the direction to increment y
        ystep = 0
        y = self.start[1]

        if self.start[1] < self.end[1]: ystep = 1
        else: ystep = -1

        # iterate over the x coordinates of the line
        for x in range(self.start[0],self.end[0]+1):
            # if the line is steep, append the coordinates with x and y swapped, otherwise append normally
            if self.steep:
                self.path.append((y,x))
            else:
                self.path.append((x,y))

            # update the error and increment y if necessary
            error += derr

            if error >= 0.5:
                y += ystep
                error -= 1.0

    # helper function to swap two values
    def swap(self,n1,n2):
        return [n2,n1]

# define a test function
def test():
    # create a Bresenham line from [8,1] to [6,4]
    l = bresenham([8,1],[6,4])
    # print the path of the line
    print(l.path)

    # create a 15x15 grid of '#' characters
    map = []
    for x in range(0,15):
        yc = []
        for y in range(0,15):
            yc.append('#')
        map.append(yc)

    # mark the path of the line with '.'
    for pos in l.path:
        map[pos[0]][pos[1]] = '.'

    # print the resulting grid
    for y in range(0,15):
        for x in range(0,15):
            print(map[x][y], end=' ')
    print()

In [145]:
# Bresenham circle algorithm
# https://www.daniweb.com/programming/software-development/threads/321181/python-bresenham-circle-arc-algorithm

def circle(radius):
    # initialize variables
    switch = 3 - (2 * radius) # Determines the first pixel position
    points = set() # A set to store the points in the circle
    x = 0
    y = radius

    # The following loop covers the first quarter of the circle and the other quarters are obtained by symmetry

    # while x is less than or equal to y (meaning that the circle is not complete yet)
    while x <= y:
        # first quarter, first octant (clockwise at 12 o'clock)
        points.add((x,-y))
        # first quarter, second octant
        points.add((y,-x))
        # second quarter, third octant
        points.add((y,x))
        # second quarter, fourth octant
        points.add((x,y))
        # third quarter, fifth octant
        points.add((-x,y))
        # third quarter, sixth octant
        points.add((-y,x))
        # fourth quarter, seventh octant
        points.add((-y,-x))
        # fourth quarter, eighth octant
        points.add((-x,-y))

        # calculate the next pixel position by updating x and y
        if switch < 0:
            # if the decision variable (switch) is negative, the current pixel is inside the circle
            switch = switch + (4 * x) + 6 # update the decision variable
        else:
            # if the decision variable is non-negative, the current pixel is outside the circle
            switch = switch + (4 * (x - y)) + 10 # update the decision variable
            y = y - 1 # move to the next row

        x = x + 1 # move to the next column

    return points # return the set of points in the circle

In [146]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [147]:
import sys
import numpy as np
import scipy
import scipy.sparse
import scipy.sparse.linalg
from scipy.sparse import csr_matrix
from imageio import imread, imsave 
from skimage.transform import resize as imresize
from skimage import io, color, transform
from skimage.color import rgb2gray
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import imageio.v2
import math
from collections import defaultdict
from PIL import Image

In [148]:
def image(filename, size):
    
    # load the image from the specified file
    img = io.imread(filename)

    # if the loaded image has 2 dimensions
    # the function assumes it is a grayscale image 
    # and creates a new 3-dimensional numpy array with the grayscale image repeated 3 times
    if img.ndim == 2:  # grayscale image
        img_rgb = np.stack((img,) * 3, axis=-1)

        # if the loaded image has 3 dimensions and the last dimension has size 3
        # the function assumes it is a color image and assigns it to the variable img_rgb
    elif img.ndim == 3 and img.shape[-1] == 3:  # color image
        img_rgb = img

        # if the loaded image has a different number of dimensions or the last dimension does not have size 3
        # the function raises a ValueError
    else:
        raise ValueError("Unsupported image format")

        # then resizes the image to the specified size using skimage.transform.resize
        # the resulting image is returned as a numpy array
    img_resized = imresize(img_rgb, (size, size), anti_aliasing=True)
    
    return img_resized

In [149]:
# The function build_arc_adjecency_matrix takes two arguments: n and radius
def build_arc_adjecency_matrix(n, radius):

    # The string "building sparse adjecency matrix" is printed to the console
    print("building sparse adjecency matrix")

    # An array hooks is created using a list comprehension that contains the coordinates of equally-spaced points on a unit circle
    # The cos and sin functions from the math module are used to calculate the coordinates
    # The range function is used to generate the indices of the points on the circle
    # The n parameter specifies the number of points
    hooks = np.array([[math.cos(np.pi*2*i/n), math.sin(np.pi*2*i/n)] for i in range(n)])

    # The hooks array is multiplied by the radius variable and then cast to an array of integers
    hooks = (radius * hooks).astype(int)

    # Three empty lists edge_codes, row_ind, and col_ind are created
    edge_codes = []
    row_ind = []
    col_ind = []

    # A nested loop is used to iterate over pairs of points in the hooks array
    # The outer loop iterates over the indices and coordinates of the first point, while the inner loop iterates over the indices and coordinates of the second point
    for i, ni in enumerate(hooks):
        for j, nj in enumerate(hooks[i+1:], start=i+1):

            # For each pair of points, a tuple representing an edge is appended to the edge_codes list
            edge_codes.append((i, j))

            # The bresenham function is called with the coordinates of the two points to generate a list of pixels that lie on the line between the two points
            # The path attribute of the resulting Line object is used to extract the list of pixels
            pixels = bresenham(ni, nj).path

            edge = []

            # For each pixel on the line, a unique integer code is generated by adding the row and column indices to the appropriate offsets and appending the resulting code to the edge list
            for pixel in pixels:
                pixel_code = (pixel[1]+radius)*(radius*2+1) + (pixel[0]+radius)
                edge.append(pixel_code)
            # The edge list is concatenated with the row_ind list, and the index of the corresponding edge is appended to the col_ind list for each element of the edge list
            row_ind += edge
            col_ind += [len(edge_codes)-1] * len(edge)

    # A sparse matrix sparse is created using the csr_matrix function from the scipy.sparse module
    # The values of the matrix are all 1.0, and the rows and columns are indexed using the row_ind and col_ind lists, respectively
    # The shape of the matrix is (2*radius+1)*(2*radius+1) by len(edge_codes)
    sparse = scipy.sparse.csr_matrix(([1.0]*len(row_ind), (row_ind, col_ind)), shape=((2*radius+1)*(2*radius+1), len(edge_codes)))

    # The function returns a tuple containing the sparse matrix sparse, the hooks array, and the edge_codes list
    return sparse, hooks, edge_codes

In [150]:
def build_circle_adjecency_matrix(radius, small_radius):
    # Print statement to indicate the start of matrix building
    print("building sparse adjecency matrix")
    
    # Empty lists to store row, column, and edge codes
    edge_codes = []
    row_ind = []
    col_ind = []
    
    # Retrieve the pixels for a small circle
    pixels = circle(small_radius)
    
    # Iterate over the centers of each circle of the given radius
    for i, cx in enumerate(range(-radius+small_radius+1, radius-small_radius-1, 1)):
        for j, cy in enumerate(range(-radius+small_radius+1, radius-small_radius-1, 1)):
            
            # Add edge code to list
            edge_codes.append((i, j))
            
            # Create empty list to store pixels in this edge
            edge = []
            
            # Iterate over pixels in the small circle and calculate their pixel codes
            for pixel in pixels:
                px, py = cx+pixel[0], cy+pixel[1]
                pixel_code = (py+radius)*(radius*2+1) + (px+radius)
                edge.append(pixel_code)
            
            # Add pixel codes and edge code to the respective lists
            row_ind += edge
            col_ind += [len(edge_codes)-1] * len(edge)
    
    # creating the edge-pixel adjacency matrix:
    # rows are indexed with pixel codes, columns are indexed with edge codes.
    sparse = scipy.sparse.csr_matrix(([1.0]*len(row_ind), (row_ind, col_ind)), shape=((2*radius+1)*(2*radius+1), len(edge_codes)))
    
    # Create empty list to store hooks
    hooks = []
    
    # Return the adjacency matrix, hooks, and edge codes
    return sparse, hooks, edge_codes

In [151]:
def build_image_vector(img, radius):
    # representing the input image as a sparse column vector of pixels:
    assert img.shape[0] == img.shape[1]  # ensure the input image is square
    img_size = img.shape[0]  # get the size of the input image
    row_ind = []  # initialize a list to store row indices
    col_ind = []  # initialize a list to store column indices
    data = []  # initialize a list to store pixel values
    for y, line in enumerate(img):  # iterate over each line (row) in the image
        for x, pixel_value in enumerate(line):  # iterate over each pixel in the line (row)
            global_x = x - img_size//2  # get the global x-coordinate of the pixel
            global_y = y - img_size//2  # get the global y-coordinate of the pixel
            pixel_code = (global_y+radius)*(radius*2+1) + (global_x+radius)  # calculate the code of the pixel
            data.append(float(pixel_value[0])) # add the pixel value to the data list
            row_ind.append(pixel_code)  # add the pixel code to the row index list
            col_ind.append(0)  # add 0 as the column index (all pixels will be in the same column)
    sparse_b = scipy.sparse.csr_matrix((data, (row_ind, col_ind)), shape=((2*radius+1)*(2*radius+1), 1))  # create a sparse column vector from the data, row indices, and column indices
    return sparse_b  # return the sparse column vector representing the input image

In [152]:
  # Function named reconstruct that takes three inputs
  # x: a numpy array that represents a compressed image
  # sparse: a scipy sparse matrix that represents the edge-pixel adjacency matrix for a given radius
  # radius: an integer representing the radius used to build the sparse matrix and image vector
def reconstruct(x, sparse, radius):
    
    # Using the sparse matrix to approximate the original image vector b by multiplying it with x, the compressed image
    # The result is a numpy array called b_approx
    b_approx = sparse.dot(x)

    # Next is to reshape b_approx into a 2D array with dimensions (2*radius+1, 2*radius+1) using the reshape method
    b_image = b_approx.reshape((2*radius+1, 2*radius+1))

    # the resulting image is clipped between 0 and 255 to ensure that all pixel values are within the valid range of a grayscale image
    b_image = np.clip(b_image, 0, 255)

    # The function returns the reconstructed image
    return b_image

In [153]:
# This function takes in four parameters:
# x: a NumPy array that contains the data to be reconstructed
# sparse: a sparse matrix containing the adjacency matrix
# radius: an integer representing the radius of the circle used to create the adjacency matrix
# filename: a string representing the filename to save the reconstructed image
def reconstruct_and_save(x, sparse, radius, output_filename):

    # Set the brightness correction factor
    brightness_correction = 1.2

    # Reconstruct the image from the sparse matrix and the input data
    b_image = reconstruct(x * brightness_correction, sparse, radius)

    # Save the reconstructed image to the output file
    imsave(output_filename, b_image)

In [154]:
def main():
    input_filename = input("Enter the input image filename: ")
    output_prefix = input("Enter the output filename prefix: ")

    # sets the values for two parameters, n and radius
    n = 180
    radius = 250

    # calls build_arc_adjacency_matrix to construct the adjacency matrix for the arc graph with n nodes and a circle with 'radius'
    sparse, hooks, edge_codes = build_arc_adjecency_matrix(n, radius)

    # loads the input image and resizes it to a square image with sides equal to 75% of the circle's diameter
    # it then constructs a sparse column vector sparse_b that represents the image
    shrinkage = 0.75
    img = image(input_filename, int(radius * 2 * shrinkage))
    sparse_b = build_image_vector(img, radius)

    # calls scipy.sparse.linalg.lsqr to solve the linear system of equations sparse * x = sparse_b
    print("solving linear system")
    result = scipy.sparse.linalg.lsqr(sparse, np.array(sparse_b.todense()).flatten())
    print("done")
    x = result[0]

    # passes the resulting x vector to reconstruct_and_save along with the adjacency matrix sparse, the radius
    # the output filename prefix to save the reconstructed image as a PNG file
    # the reconstructed image is also multiplied by a brightness correction factor of 1.2 before being saved
    reconstruct_and_save(x, sparse, radius, output_prefix+"-STRING.png")

main()

Enter the input image filename: drive/My Drive/ComputerVision/Final Project/Star.jpeg
Enter the output filename prefix: drive/My Drive/ComputerVision/Final Project/ULTRASTAR.jpg
building sparse adjecency matrix
solving linear system


done
